In [5]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm.auto import tqdm

df = pd.read_csv("하이닉스 power1_data.csv", thousands=',', encoding='utf-8')
# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = (df['종가'].shift(-1) - df['종가'])/ df['종가'] *100
df['target'] = df['next_day_return'].apply(lambda x: 2 if x > 0.25 else 1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

#df=df.iloc[-300:,:]
X = df.drop(['날짜','target','next_day_return'], axis=1) 
y = df['target']


# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_7_features = [feature for feature, importance in features_rf_sorted[:7]]

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)dp
X_top8 = X[top_7_features]
y = df['target']  # y 값을 0과 1로 조정

# 데이터를 학습 세트와 테스트 세트로 분할
# Define y as before
y = df['target']

# Precision 점수를 저장할 리스트 초기화
precision_scores = []

# tqdm을 사용하여 random_state에 대한 반복 실행
for state in tqdm(range(1, 101), desc='Evaluating Random States'):
    X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=state)
    
    svc_model = GradientBoostingClassifier(random_state=state)
    svc_model.fit(X_train, y_train)
    y_pred = svc_model.predict(X_test)
    
    precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
    precision_scores.append({'Random_State': state, 'Precision': precision})

# 리스트를 DataFrame으로 변환
precision_scores_df = pd.DataFrame(precision_scores)

# DataFrame 출력 설정 변경
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 결과 출력
precision_scores_df_sorted = precision_scores_df.sort_values(by='Precision', ascending=False)
precision_scores_df_sorted

Evaluating Random States:   0%|          | 0/100 [00:00<?, ?it/s]

,Random_State,Precision
89,90,0.588979
8,9,0.579197
53,54,0.578123
76,77,0.573350
3,4,0.571653
37,38,0.569822
97,98,0.569132
92,93,0.567344
98,99,0.567010
59,60,0.564892


In [7]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm.auto import tqdm

df = pd.read_csv("하이닉스 power1_data.csv", thousands=',', encoding='utf-8')
# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = (df['종가'].shift(-1) - df['종가'])/ df['종가'] *100
df['target'] = df['next_day_return'].apply(lambda x: 2 if x > 0.25 else 1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

#df=df.iloc[-300:,:]
X = df.drop(['날짜','target','next_day_return'], axis=1) 
y = df['target']


# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_7_features = [feature for feature, importance in features_rf_sorted[:7]]

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)dp
X_top8 = X[top_7_features]
y = df['target']  # y 값을 0과 1로 조정

# 데이터를 학습 세트와 테스트 세트로 분할
# Define y as before
y = df['target']

# Precision 점수를 저장할 리스트 초기화
precision_scores = []

# tqdm을 사용하여 random_state에 대한 반복 실행
for state in tqdm(range(1, 101), desc='Evaluating Random States'):
    X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=state)
    
    svc_model =LinearRegression(random_state=state)
    svc_model.fit(X_train, y_train)
    y_pred = svc_model.predict(X_test)
    
    precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
    precision_scores.append({'Random_State': state, 'Precision': precision})

# 리스트를 DataFrame으로 변환
precision_scores_df = pd.DataFrame(precision_scores)

# DataFrame 출력 설정 변경
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 결과 출력
precision_scores_df_sorted = precision_scores_df.sort_values(by='Precision', ascending=False)
precision_scores_df_sorted

Evaluating Random States:   0%|          | 0/100 [00:00<?, ?it/s]

,Random_State,Precision
17,18,0.587898
37,38,0.581531
58,59,0.580707
67,68,0.580393
19,20,0.580294
7,8,0.573761
43,44,0.573712
97,98,0.572961
88,89,0.570302
5,6,0.567088
